# 💬 | JAC ChatBot

## 💾 | Cargar datos

In [1]:
# Importar el cargador de documentos PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
# Cargar el documento
loader = PyPDFLoader("../docs/constitucion_politica.pdf")
# Obtener el documento en bruto
raw_document = loader.load()
# Imprimir el tamaño del documento en bruto
print("Size of the raw document: ", len(raw_document))
# Imprimir el primer caracter del documento en bruto
raw_document[0]

/Users/laasanchezgi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Size of the raw document:  120


Document(metadata={'source': '../docs/constitucion_politica.pdf', 'page': 0}, page_content=' \nConvenciones: \n \n- Los textos encerrados entre los símbolos <...> fueron agregados por el \neditor, con el único propósito de facilitar la consulta de este documento legal. \nDichos textos no corresponden a las ediciones oficiales de la Constitución \nPolítica de 1991, publicadas en las Gacetas Constitucionales Nos. 114, 116 y \n125  de  1991,  ni  de  sus  modificaciones  a  través  de  Actos  Legislativos \npublicados en diferentes Diarios Oficiales. \n \n  \nPREAMBULO \n \nEl pueblo de Colombia, \n \nen ejercicio de su poder soberano, representado por sus delegatarios a la \nAsamblea Nacional Constituyente, invocando la protección de Dios, y con el fin \nde fortalecer la unidad de la Nación y asegurar a sus integrantes la vida, la \nconvivencia, el trabajo, la justicia, la igualdad, el conocimiento, la libertad y la \npaz, dentro de un marco jurídico, democrático y participativo que gara

## 🔪 | Generar chunks a partir del documento

In [2]:
# Importar el separador de texto RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Crear una instancia del separador de texto
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # Tamaño de los fragmentos
    chunk_overlap=100 # Traslape de los fragmentos
)
# Separar el documento en fragmentos
documents = splitter.split_documents(raw_document)
# Imprimir el tamaño de los documentos separados
print("Size of the splitted documents: ", len(documents))
# Imprimir el primer fragmento
documents[0]

Size of the splitted documents:  375


Document(metadata={'source': '../docs/constitucion_politica.pdf', 'page': 0}, page_content='Convenciones: \n \n- Los textos encerrados entre los símbolos <...> fueron agregados por el \neditor, con el único propósito de facilitar la consulta de este documento legal. \nDichos textos no corresponden a las ediciones oficiales de la Constitución \nPolítica de 1991, publicadas en las Gacetas Constitucionales Nos. 114, 116 y \n125  de  1991,  ni  de  sus  modificaciones  a  través  de  Actos  Legislativos \npublicados en diferentes Diarios Oficiales. \n \n  \nPREAMBULO \n \nEl pueblo de Colombia, \n \nen ejercicio de su poder soberano, representado por sus delegatarios a la \nAsamblea Nacional Constituyente, invocando la protección de Dios, y con el fin \nde fortalecer la unidad de la Nación y asegurar a sus integrantes la vida, la \nconvivencia, el trabajo, la justicia, la igualdad, el conocimiento, la libertad y la \npaz, dentro de un marco jurídico, democrático y participativo que garanti

## 💰 | Contar los tokens y generar un costo estimado del embedding

In [3]:
import tiktoken
encoder = tiktoken.encoding_for_model("text-embedding-ada-002")
doc_tokens = [len(encoder.encode(doc.page_content)) for doc in documents]
total_tokens = sum(doc_tokens)
cost = (total_tokens/1000) * 0.0001
print(f"Total tokens: {total_tokens} - cost: ${cost:.6f}")

Total tokens: 90482 - cost: $0.009048


## 💽 | Almacenar los embeddings en la base de datos vectorial

In [4]:
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# Asignar las variables de entorno a os.environ
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY

# Crear una instancia de AzureOpenAIEmbeddings
embedding = AzureOpenAIEmbeddings(
    deployment="text-embedding-ada-002",# Modelo de embeddings
    chunk_size = 1 # Tamaño de los fragmentos
    )
# Crear una instancia de Chroma
db_cp = Chroma(
    persist_directory="chroma_db_cp", # Directorio de persistencia
    embedding_function=embedding # Función de embeddings
    )
# Obtener el contenido de la base de datos
db_cp.get() 

## ⛓️ | Crear una cadena QA

In [23]:
# Importar el prompt template
from langchain_core.prompts import PromptTemplate
# Importar el parser de salida
from langchain_core.output_parsers import StrOutputParser 

# Crear una pregunta
question_cp = "¿Cuales son los fines esenciales del Estado Colombiano?"
# Buscar documentos similares
context_docs = db_cp.similarity_search(question_cp)
# Prompt template
prompt = PromptTemplate(
    template=""""
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    <context>
    {context}
    </context>

    Question: {question}
    Helpful Answer, formatted in markdown:
    """,
    input_variables=["context", "question"]
)

# Crear una instancia del parser de salida
llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT ,
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt-4",
)

# Crear la cadena
qa_chain = prompt | llm | StrOutputParser() # Cadena de lo que ejecuta

# Llamar a la cadena
result = qa_chain.invoke({
    "question": question_cp,
    "context": "\n".join([doc.page_content for doc in context_docs])
})

# Imprimir el resultado a la pregunta
print(result)

Los fines esenciales del Estado Colombiano están consagrados en la Constitución Política de Colombia de 1991. Según el Artículo 2 de la Constitución, los fines esenciales del Estado son:

1. **Servir a la comunidad**: El Estado está al servicio de la comunidad, trabajando para promover el bienestar general y mejorar la calidad de vida de la población.
2. **Garantizar la efectividad de los principios, derechos y deberes consagrados en la Constitución**: El Estado debe asegurar que todos los principios y derechos fundamentales, así como los deberes de los ciudadanos, sean respetados y cumplidos.
3. **Facilitar la participación de todos en las decisiones que los afectan y en la vida económica, política, administrativa y cultural de la Nación**: El Estado debe promover la inclusión y participación activa de todos los ciudadanos en los diferentes aspectos de la gestión pública y la vida nacional.
4. **Defender la independencia nacional, mantener la integridad territorial y asegurar la convi